# TensorFlow Example: Word Embeddings

In this example, we see how we can try a classification neural network using TensorFlow, and extract word embeddings out of it.

# Imports

In [1]:
import tensorflow as tf
print(tf.__version__)

2022-10-18 15:03:00.867092: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-18 15:03:00.917778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-18 15:03:00.917796: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.1


### We NEED to use TensorFlow 2.3.0+

We will also make use of TensorFlow datasets. If you don't have it installed yet, you can do it running:

```
!pip install -q tensorflow-datasets
```

In a dedicated jupyter notebook cell.

# Hyperparameters

In [3]:
vocab_size = 10000
embedding_dim = 16
max_length = 80
trunc_type='post'
oov_tok = "<OOV>"

# Load data

In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

2022-10-18 15:04:05.968023: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling /home/lapbeer/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZT15YH/imdb_reviews-test.t…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /home/lapbeer/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZT15YH/imdb_reviews-unsupe…

Dataset imdb_reviews downloaded and prepared to /home/lapbeer/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2022-10-18 15:05:00.022776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-18 15:05:00.022797: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-18 15:05:00.022812: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lapbeer): /proc/driver/nvidia/version does not exist
2022-10-18 15:05:00.023626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [13]:
# The reverse_word_index dictionaty gives the index of each word in reverse, to facilitete decoding.
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text): 
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print('Original training sentence:')
print(training_sentences[3])
print('___________________________')
print('Padded sentence:')
print(padded[3])
print('___________________________')
print('Decoded sentence:')
print(decode_review(padded[3]))

Original training sentence:
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.
___________________________
Padded sentence:
[  12    7    2  241    5   20   16    4 8776 2707 2653   52    2  358
    5    2  180   68  138 1401   17   92  203  968   15   23    1   81
    4  192 3109 3037    3    1   16    4  376    5  632  387  352   37
 6355    3 5559 1928   15  208 8596 3433    2  112  365   48   24   55
    1    6 1657   55 1818 4312   41    4 2263    3 1908    1  141  160
  780  111   31   92  116    4  221   20    9  173]
___________________________
Decoded sentence:
this is t

# Model Training

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 16)            160000    
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 7686      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 167,693
Trainable params: 167,693
Non-trainable params: 0
_________________________________________________________________


In [9]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 3s 3ms/step - loss: 0.5245 - accuracy: 0.7209 - val_loss: 0.4020 - val_accuracy: 0.8144
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2794 - accuracy: 0.8871 - val_loss: 0.4234 - val_accuracy: 0.8091
Epoch 3/10
782/782 [==============================] - 3s 3ms/step - loss: 0.1147 - accuracy: 0.9686 - val_loss: 0.5340 - val_accuracy: 0.7926
Epoch 4/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0302 - accuracy: 0.9961 - val_loss: 0.6411 - val_accuracy: 0.7876
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0080 - accuracy: 0.9994 - val_loss: 0.7239 - val_accuracy: 0.7859
Epoch 6/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.7872 - val_accuracy: 0.7876
Epoch 7/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.8445 - val_accuracy: 0.7885
Epoch 

In [10]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


### You should get:
```
(10000, 16)
```
Which makes sense.
Remember, we got 10000 words in our corpus. And 16 is an hyperparameter set previously. This will set the size of the word embeddings.


## Exporting Word Embeddings

Now we will save the trained word embeddings (remember, they are the weights of the neural network).

In [11]:
import io

out_v = io.open('vecs_imdb.tsv', 'w', encoding='utf-8')
out_m = io.open('meta_imdb.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num] #reverses the word to get its index
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

If you are running this on Google Colab, you will need to download the generated files. You can do it running:
```Python
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs_imdb.tsv')
  files.download('meta_imdb.tsv')
```